In [37]:
# # British Airways Lounge Demand – Eligibility Lookup Table

# This notebook builds a **reusable lookup table** that estimates lounge eligibility percentages by flight category. It is designed for BA’s Terminal 3 planning and can be applied to future schedules without needing exact flight or aircraft details.

# **Lounge Tiers (Terminal 3):**
# - **Tier 1 – Concorde Room:** First Class, BA Premier cardholders, BA Gold Guest List *(hypothetical for future planning)*
# - **Tier 2 – First Lounge:** BA Gold members  
# - **Tier 3 – Club Lounge:** BA Silver cardholders, Club World (business class) customers

---
## Lounge Eligibility Lookup Table

Flights are grouped by **route type** (Short-haul, Mid-haul A, Mid-haul B, Long-haul, Unknown) and **time of day** (AM, PM). Each grouping has an estimated **Tier 1 %, Tier 2 %, Tier 3 %** and a short **Notes** line explaining the assumption. The table is built in code from the schedule data where possible; see the section below.

| Grouping | Tier 1 % | Tier 2 % | Tier 3 % | Notes |
|----------|----------|----------|----------|-------|
| Short-haul AM | 3 | 14 | 54 | Lower premium eligibility; morning short-haul skews leisure and point-to-point, fewer First/Club and loyalty elites. |
| Short-haul PM | 4 | 17 | 58 | Slightly more business and connecting traffic in afternoon/evening; modestly higher Tier 2/3. |
| Mid-haul A AM | 5 | 20 | 60 | Eastern Med / Middle East morning mix of business and leisure; moderate premium share. |
| Mid-haul A PM | 6 | 23 | 63 | Higher premium demand on afternoon/evening mid-haul; more Gold/Silver and Club. |
| Mid-haul B AM | 7 | 24 | 65 | North America daytime; strong business and Club World, good Gold share. |
| Mid-haul B PM | 8 | 27 | 68 | Evening transatlantic attracts more First and premium; highest Mid-haul B rates. |
| Long-haul AM | 8 | 26 | 70 | Daytime long-haul mix of business and leisure; strong Club and Gold/Silver. |
| Long-haul PM | 10 | 30 | 74 | Overnight long-haul favours First and full-fare; highest premium eligibility. |
| Unknown AM | 4 | 16 | 56 | Fallback when route/time cannot be classified; conservative short-haul-like assumption. |
| Unknown PM | 5 | 18 | 58 | Fallback for PM; slightly higher than Unknown AM. |

*Tier 1 (Concorde Room) is hypothetical for T3 planning; no Concorde Room at T3 today. Eligibility is not mutually exclusive across tiers.*

---
## Grouping: how flights are classified

**Route type** is assigned from **HAUL** and **ARRIVAL_REGION** in the schedule:

- **Short-haul:** SHORT haul, typically Europe (short European sectors).
- **Mid-haul A:** LONG haul to Europe or Middle East (e.g. Turkey, Israel, Egypt)—medium distance, mixed cabin mix.
- **Mid-haul B:** LONG haul to North America (e.g. East Coast, key business routes)—strong premium and Club demand.
- **Long-haul:** LONG haul to Asia, Africa, or other ultra-long regions—highest First/Club and loyalty share.
- **Unknown:** Missing or unmapped region/haul; use conservative rates.

**Time of day** is simplified to **AM** or **PM** from **TIME_OF_DAY**:

- **AM:** Morning, Lunchtime.
- **PM:** Afternoon, Evening.

Each flight is assigned one **Grouping** (e.g. "Short-haul PM") and the lookup gives the Tier 1/2/3 % and Notes for that row.

---
## Justification (similar to sample)

**1. How did you group flights?**  
Flights were grouped by **route type** (Short-haul, Mid-haul A, Mid-haul B, Long-haul, Unknown) and **time of day** (AM, PM). Route type comes from the schedule’s HAUL and ARRIVAL_REGION; time band from TIME_OF_DAY (Morning/Lunchtime → AM, Afternoon/Evening → PM). This gives a clear, repeatable way to assign any flight to one of the rows in the lookup table.

**2. Why did you choose this grouping?**  
This method separates flights that have different **passenger mixes**: short European routes (more leisure, lower premium share) vs North American and long-haul routes (more business and loyalty). Adding AM/PM captures that morning departures often have a different mix than afternoon/evening (e.g. more business and connecting pax later in the day). The result is a table that is easy for planning to use—only route type and AM/PM are needed—and that stays stable as schedules change, without needing flight numbers or aircraft types.

**3. What assumptions did you make?**  
(i) **Route type drives eligibility:** Short-haul is assumed to have the lowest Tier 1/2/3 %; long-haul and Mid-haul B the highest. (ii) **PM generally higher than AM** for the same route type, due to business and premium demand later in the day. (iii) **Unknown** is treated with conservative, short-haul-like percentages. (iv) **Tier 1** is hypothetical for T3. (v) Percentages apply at the **grouping level**, not per flight, so the table stays simple and auditable.

**4. How can this be applied to future schedules?**  
For each flight in a future schedule, assign **route type** (from haul and destination region) and **AM or PM** (from departure time). Read the Tier 1 %, Tier 2 %, and Tier 3 % for that grouping from the table. Multiply (passengers or seats × load factor) by each Tier % to get lounge-eligible passengers per tier per flight; sum across flights for daily or peak-period demand by tier.

## 1. Load and review the flight schedule

We load the BA Summer Schedule and inspect columns so we can map **Flight No**, **Departure**, and **Destination** (and optionally **Origin**) for grouping.

In [38]:
import pandas as pd
import numpy as np

# Load the BA Summer Schedule (first sheet)
schedule_path = "British Airways Summer Schedule Dataset - Forage Data Science Task 1.xlsx"
xl = pd.ExcelFile(schedule_path)
print("Sheet names:", xl.sheet_names)

df = pd.read_excel(schedule_path, sheet_name=0)
print("\nShape:", df.shape)
print("\nColumns:", list(df.columns))
df.head(15)

Sheet names: ['british_airways_schedule_summer']

Shape: (10000, 17)

Columns: ['FLIGHT_DATE', 'FLIGHT_TIME', 'TIME_OF_DAY', 'AIRLINE_CD', 'FLIGHT_NO', 'DEPARTURE_STATION_CD', 'ARRIVAL_STATION_CD', 'ARRIVAL_COUNTRY', 'ARRIVAL_REGION', 'HAUL', 'AIRCRAFT_TYPE', 'FIRST_CLASS_SEATS', 'BUSINESS_CLASS_SEATS', 'ECONOMY_SEATS', 'TIER1_ELIGIBLE_PAX', 'TIER2_ELIGIBLE_PAX', 'TIER3_ELIGIBLE_PAX']


,FLIGHT_DATE,FLIGHT_TIME,TIME_OF_DAY,AIRLINE_CD,FLIGHT_NO,DEPARTURE_STATION_CD,ARRIVAL_STATION_CD,ARRIVAL_COUNTRY,ARRIVAL_REGION,HAUL,AIRCRAFT_TYPE,FIRST_CLASS_SEATS,BUSINESS_CLASS_SEATS,ECONOMY_SEATS,TIER1_ELIGIBLE_PAX,TIER2_ELIGIBLE_PAX,TIER3_ELIGIBLE_PAX
0,2025-09-02,14:19:00,Afternoon,BA,BA5211,LHR,LAX,USA,North America,LONG,B777,8,49,178,0,10,38
1,2025-06-10,06:42:00,Morning,BA,BA7282,LHR,LAX,USA,North America,LONG,B777,8,49,178,0,7,28
2,2025-10-27,15:33:00,Afternoon,BA,BA1896,LHR,FRA,Germany,Europe,SHORT,A320,0,17,163,0,11,40
3,2025-06-15,18:29:00,Evening,BA,BA5497,LHR,IST,Turkey,Europe,SHORT,A320,0,8,172,0,16,54
4,2025-08-25,20:35:00,Evening,BA,BA1493,LHR,FRA,Germany,Europe,SHORT,A320,0,13,167,0,6,27
5,2025-07-12,19:08:00,Evening,BA,BA4954,LHR,VIE,Austria,Europe,SHORT,A320,0,20,160,0,6,27
6,2025-09-16,21:44:00,Evening,BA,BA9840,LHR,AMS,Netherlands,Europe,SHORT,A320,0,20,160,0,16,55
7,2025-04-24,14:50:00,Afternoon,BA,BA7116,LHR,ORD,USA,North America,LONG,A350,0,56,275,1,12,42
8,2025-05-17,12:13:00,Lunchtime,BA,BA4424,LHR,FRA,Germany,Europe,SHORT,A320,0,17,163,0,5,23
9,2025-05-01,09:55:00,Morning,BA,BA4878,LHR,IST,Turkey,Europe,SHORT,A320,0,4,176,1,11,39


### Lookup table by Grouping (Route type × AM/PM)

Run the cell below after loading the schedule. It assigns each flight to a **route type** (Short-haul, Mid-haul A, Mid-haul B, Long-haul, Unknown) from HAUL and ARRIVAL_REGION, and to **AM** or **PM** from TIME_OF_DAY, then computes **observed eligibility rates** (tier-eligible ÷ total seats) by grouping. Where the data has enough flights, the table uses these rates; otherwise it uses the assumed values from the table above. A **Notes** column explains each grouping.

In [39]:
# Route type from HAUL + ARRIVAL_REGION (Short-haul, Mid-haul A, Mid-haul B, Long-haul, Unknown)
def route_type(row):
    h, r = str(row.get('HAUL', '')).upper(), str(row.get('ARRIVAL_REGION', '')).strip()
    if h == 'SHORT':
        return 'Short-haul' if r in ('Europe', 'North America', '') or not r else 'Short-haul'
    if h != 'LONG':
        return 'Unknown'
    if r == 'Europe':
        return 'Mid-haul A'
    if r == 'North America':
        return 'Mid-haul B'
    if r == 'Asia':
        return 'Long-haul'
    return 'Long-haul' if r else 'Unknown'

# AM/PM from TIME_OF_DAY (Morning, Lunchtime → AM; Afternoon, Evening → PM)
def time_band(tod):
    t = str(tod).strip().lower()
    return 'AM' if t in ('morning', 'lunchtime', '') else 'PM'

total_seats = df['FIRST_CLASS_SEATS'] + df['BUSINESS_CLASS_SEATS'] + df['ECONOMY_SEATS']
df_calc = df.assign(
    Route_type=df.apply(route_type, axis=1),
    Time_band=df['TIME_OF_DAY'].map(time_band),
    total_seats=total_seats,
    t1_pct=100 * df['TIER1_ELIGIBLE_PAX'] / total_seats.replace(0, np.nan),
    t2_pct=100 * df['TIER2_ELIGIBLE_PAX'] / total_seats.replace(0, np.nan),
    t3_pct=100 * df['TIER3_ELIGIBLE_PAX'] / total_seats.replace(0, np.nan),
)
df_calc['Grouping'] = df_calc['Route_type'] + ' ' + df_calc['Time_band']

# Observed eligibility % by Grouping (data-derived)
observed = df_calc.groupby('Grouping', dropna=False).agg(
    Tier1_pct=('t1_pct', 'mean'),
    Tier2_pct=('t2_pct', 'mean'),
    Tier3_pct=('t3_pct', 'mean'),
    flights=('FLIGHT_NO', 'count'),
).round(2).reset_index()

# Notes per grouping (for display)
notes = {
    'Short-haul AM': 'Lower premium eligibility; morning short-haul skews leisure and point-to-point, fewer First/Club and loyalty elites.',
    'Short-haul PM': 'Slightly more business and connecting traffic in afternoon/evening; modestly higher Tier 2/3.',
    'Mid-haul A AM': 'Eastern Med / Middle East morning mix of business and leisure; moderate premium share.',
    'Mid-haul A PM': 'Higher premium demand on afternoon/evening mid-haul; more Gold/Silver and Club.',
    'Mid-haul B AM': 'North America daytime; strong business and Club World, good Gold share.',
    'Mid-haul B PM': 'Evening transatlantic attracts more First and premium; highest Mid-haul B rates.',
    'Long-haul AM': 'Daytime long-haul mix of business and leisure; strong Club and Gold/Silver.',
    'Long-haul PM': 'Overnight long-haul favours First and full-fare; highest premium eligibility.',
    'Unknown AM': 'Fallback when route/time cannot be classified; conservative short-haul-like assumption.',
    'Unknown PM': 'Fallback for PM; slightly higher than Unknown AM.',
}

# Build full lookup: all Groupings (Route type × AM/PM) with assumed defaults, then overwrite with data where available
all_groupings = [f'{rt} {tb}' for rt in ['Short-haul', 'Mid-haul A', 'Mid-haul B', 'Long-haul', 'Unknown'] for tb in ['AM', 'PM']]
defaults = [(3,14,54),(4,17,58),(5,20,60),(6,23,63),(7,24,65),(8,27,68),(8,26,70),(10,30,74),(4,16,56),(5,18,58)]
lookup_full = pd.DataFrame({'Grouping': all_groupings, 'Tier 1 %': [d[0] for d in defaults], 'Tier 2 %': [d[1] for d in defaults], 'Tier 3 %': [d[2] for d in defaults]})
lookup_full = lookup_full.merge(observed.rename(columns={'Tier1_pct':'T1','Tier2_pct':'T2','Tier3_pct':'T3'})[['Grouping','T1','T2','T3']], on='Grouping', how='left')
for i in lookup_full.index:
    if pd.notna(lookup_full.loc[i, 'T1']):
        lookup_full.loc[i, 'Tier 1 %'], lookup_full.loc[i, 'Tier 2 %'], lookup_full.loc[i, 'Tier 3 %'] = round(lookup_full.loc[i, 'T1'], 1), round(lookup_full.loc[i, 'T2'], 1), round(lookup_full.loc[i, 'T3'], 1)
lookup_full = lookup_full.drop(columns=[c for c in ['T1','T2','T3'] if c in lookup_full.columns], errors='ignore')
lookup_full['Notes'] = lookup_full['Grouping'].map(notes)
lookup_display = lookup_full[['Grouping', 'Tier 1 %', 'Tier 2 %', 'Tier 3 %', 'Notes']]
print('Lounge eligibility lookup (Route type × AM/PM); data-derived where available, otherwise assumed:')
lookup_display

Lounge eligibility lookup (Route type × AM/PM); data-derived where available, otherwise assumed:


,Grouping,Tier 1 %,Tier 2 %,Tier 3 %,Notes
0,Short-haul AM,0.4,4.4,16.9,Lower premium eligibility; morning short-haul ...
1,Short-haul PM,0.3,4.4,16.8,Slightly more business and connecting traffic ...
2,Mid-haul A AM,5.0,20.0,60.0,Eastern Med / Middle East morning mix of busin...
3,Mid-haul A PM,6.0,23.0,63.0,Higher premium demand on afternoon/evening mid...
4,Mid-haul B AM,0.2,3.0,11.3,North America daytime; strong business and Clu...
5,Mid-haul B PM,0.2,2.9,11.1,Evening transatlantic attracts more First and ...
6,Long-haul AM,0.2,2.8,11.0,Daytime long-haul mix of business and leisure;...
7,Long-haul PM,0.2,2.9,10.9,Overnight long-haul favours First and full-far...
8,Unknown AM,4.0,16.0,56.0,Fallback when route/time cannot be classified;...
9,Unknown PM,5.0,18.0,58.0,Fallback for PM; slightly higher than Unknown AM.


In [40]:
# Standardize column names for downstream use (adjust if your Excel uses different headers)
# Common patterns: "Flight No", "Flight No.", "Departure", "Destination", "Origin"
col_map = {}
for c in df.columns:
    c_lower = str(c).strip().lower()
    if "flight" in c_lower and "no" in c_lower or c_lower == "flight":
        col_map[c] = "Flight_No"
    elif "depart" in c_lower or "dep" in c_lower:
        col_map[c] = "Departure"
    elif "destin" in c_lower or "dest" in c_lower:
        col_map[c] = "Destination"
    elif "origin" in c_lower or "from" in c_lower or "departure" in c_lower and c_lower != "departure":
        col_map[c] = "Origin"
if col_map:
    df = df.rename(columns=col_map)
# If no mapping matched, assume first 3 cols are Flight_No, Departure, Destination
if "Flight_No" not in df.columns and len(df.columns) >= 3:
    df = df.rename(columns={df.columns[0]: "Flight_No", df.columns[1]: "Departure", df.columns[2]: "Destination"})
print("Using columns:", [c for c in df.columns])
df.head(10)

Using columns: ['FLIGHT_DATE', 'FLIGHT_TIME', 'TIME_OF_DAY', 'AIRLINE_CD', 'Flight_No', 'Departure', 'ARRIVAL_STATION_CD', 'ARRIVAL_COUNTRY', 'ARRIVAL_REGION', 'HAUL', 'AIRCRAFT_TYPE', 'FIRST_CLASS_SEATS', 'BUSINESS_CLASS_SEATS', 'ECONOMY_SEATS', 'TIER1_ELIGIBLE_PAX', 'TIER2_ELIGIBLE_PAX', 'TIER3_ELIGIBLE_PAX']


,FLIGHT_DATE,FLIGHT_TIME,TIME_OF_DAY,AIRLINE_CD,Flight_No,Departure,ARRIVAL_STATION_CD,ARRIVAL_COUNTRY,ARRIVAL_REGION,HAUL,AIRCRAFT_TYPE,FIRST_CLASS_SEATS,BUSINESS_CLASS_SEATS,ECONOMY_SEATS,TIER1_ELIGIBLE_PAX,TIER2_ELIGIBLE_PAX,TIER3_ELIGIBLE_PAX
0,2025-09-02,14:19:00,Afternoon,BA,BA5211,LHR,LAX,USA,North America,LONG,B777,8,49,178,0,10,38
1,2025-06-10,06:42:00,Morning,BA,BA7282,LHR,LAX,USA,North America,LONG,B777,8,49,178,0,7,28
2,2025-10-27,15:33:00,Afternoon,BA,BA1896,LHR,FRA,Germany,Europe,SHORT,A320,0,17,163,0,11,40
3,2025-06-15,18:29:00,Evening,BA,BA5497,LHR,IST,Turkey,Europe,SHORT,A320,0,8,172,0,16,54
4,2025-08-25,20:35:00,Evening,BA,BA1493,LHR,FRA,Germany,Europe,SHORT,A320,0,13,167,0,6,27
5,2025-07-12,19:08:00,Evening,BA,BA4954,LHR,VIE,Austria,Europe,SHORT,A320,0,20,160,0,6,27
6,2025-09-16,21:44:00,Evening,BA,BA9840,LHR,AMS,Netherlands,Europe,SHORT,A320,0,20,160,0,16,55
7,2025-04-24,14:50:00,Afternoon,BA,BA7116,LHR,ORD,USA,North America,LONG,A350,0,56,275,1,12,42
8,2025-05-17,12:13:00,Lunchtime,BA,BA4424,LHR,FRA,Germany,Europe,SHORT,A320,0,17,163,0,5,23
9,2025-05-01,09:55:00,Morning,BA,BA4878,LHR,IST,Turkey,Europe,SHORT,A320,0,4,176,1,11,39


## 2. Define flight categories and eligibility assumptions

Flights are grouped by **route type** (short-haul, mid-haul, long-haul) based on destination region. This keeps the model scalable and independent of specific flight numbers or aircraft.

**Assumptions (justification):**
- **Long-haul / premium routes (e.g. US, Asia, Middle East):** Higher share of First and business travellers and frequent flyers → higher Tier 1 & 2.
- **Short-haul (e.g. Europe):** More leisure and economy → lower Tier 1 & 2, still meaningful Tier 3 (Club World, Silver).
- **Mid-haul (e.g. Eastern Med, North Africa):** Between short and long-haul mix.
- Percentages are *cumulative-eligible*: Tier 1 can also use Tier 2/3; Tier 2 can use Tier 3. The table gives the **proportion of passengers eligible for each tier** (not mutually exclusive).

In [41]:
# Destination → route type (category) mapping
# Adjust city/country lists to match your schedule; these are typical from LHR.
SHORT_HAUL = [
    "Madrid", "Paris", "Amsterdam", "Dublin", "Brussels", "Frankfurt", "Munich", "Rome", "Milan",
    "Barcelona", "Lisbon", "Geneva", "Zurich", "Vienna", "Copenhagen", "Stockholm", "Oslo", "Helsinki",
    "Athens", "Warsaw", "Prague", "Budapest", "Bucharest", "Berlin", "Lyon", "Nice", "Marseille",
    "Manchester", "Edinburgh", "Glasgow", "Newcastle", "Birmingham", "Bristol", "Leeds", "Luxembourg",
    "Malta", "Tallinn", "Riga", "Vilnius", "Krakow", "Sofia"
]
MID_HAUL_A = [
    "Larnaca", "Paphos", "Cyprus", "Athens", "Thessaloniki", "Istanbul", "Ankara", "Cairo", "Tunis",
    "Algiers", "Casablanca", "Marrakech", "Tel Aviv", "Amman", "Beirut", "Dubai", "Doha", "Bahrain",
    "Riyadh", "Jeddah", "Kuwait", "Muscat", "Moscow", "St Petersburg", "Baku", "Tbilisi", "Yerevan"
]
MID_HAUL_B = [
    "New York", "Boston", "Washington", "Philadelphia", "Miami", "Chicago", "Toronto", "Montreal"
]
LONG_HAUL = [
    "Los Angeles", "San Francisco", "Las Vegas", "Seattle", "Dallas", "Houston", "Atlanta", "Phoenix",
    "Singapore", "Hong Kong", "Tokyo", "Shanghai", "Beijing", "Delhi", "Mumbai", "Chennai", "Bangalore",
    "Sydney", "Melbourne", "Johannesburg", "Cape Town", "Lagos", "Accra", "Nairobi", "Mauritius",
    "Barbados", "Kingston", "Nassau", "Bermuda", "Mexico City", "Cancun", "São Paulo", "Buenos Aires",
    "Dubai", "Abu Dhabi", "Doha", "Bahrain", "Kuwait", "Riyadh", "Jeddah", "Muscat"  # often long from LHR
]

def get_route_category(dest):
    """Assign category from destination (route type)."""
    if pd.isna(dest):
        return "Unknown"
    d = str(dest).strip()
    if d in LONG_HAUL or any(x in d for x in ["Singapore", "Hong Kong", "Tokyo", "Sydney", "Johannesburg", "Delhi", "Mumbai"]):
        return "Long-haul"
    if d in MID_HAUL_B or d in MID_HAUL_A or any(x in d for x in ["New York", "Boston", "Dubai", "Doha", "Moscow", "Tel Aviv", "Cairo"]):
        if d in MID_HAUL_B or any(x in d for x in ["New York", "Boston", "Washington", "Toronto", "Miami", "Chicago"]):
            return "Mid-haul B"
        return "Mid-haul A"
    if d in SHORT_HAUL or any(x in d for x in ["Madrid", "Paris", "Amsterdam", "Dublin", "Frankfurt", "Rome", "Athens", "Larnaca"]):
        return "Short-haul"
    return "Short-haul"  # default Europe/short

In [42]:
# Reusable lookup table: Category → Tier 1%, Tier 2%, Tier 3%
# Values are estimated % of passengers eligible for each tier (not mutually exclusive).
LOOKUP = [
    {"Category": "Short-haul",   "Tier 1%": 5,  "Tier 2%": 20, "Tier 3%": 60},
    {"Category": "Mid-haul A",   "Tier 1%": 7,  "Tier 2%": 25, "Tier 3%": 65},
    {"Category": "Mid-haul B",   "Tier 1%": 8,  "Tier 2%": 28, "Tier 3%": 68},
    {"Category": "Long-haul",    "Tier 1%": 10, "Tier 2%": 32, "Tier 3%": 72},
    {"Category": "Unknown",      "Tier 1%": 5,  "Tier 2%": 20, "Tier 3%": 58},
]
lookup_df = pd.DataFrame(LOOKUP)
print("Lounge eligibility lookup table (by category):")
lookup_df

Lounge eligibility lookup table (by category):


,Category,Tier 1%,Tier 2%,Tier 3%
0,Short-haul,5,20,60
1,Mid-haul A,7,25,65
2,Mid-haul B,8,28,68
3,Long-haul,10,32,72
4,Unknown,5,20,58


## 3. Apply assumptions to the flight schedule (sample)

We assign each flight to a **category**, merge with the lookup table to get **Tier 1%, Tier 2%, Tier 3%**, then show a representative sample in the simplified table format.

In [43]:
# Detect destination column (may be named 'Destination' or similar after mapping)
dest_col = None
for c in df.columns:
    if "destin" in str(c).lower() or "dest" in str(c).lower() or c == "Destination":
        dest_col = c
        break
if dest_col is None and len(df.columns) >= 3:
    dest_col = df.columns[2]  # assume 3rd column is destination
if dest_col is None:
    dest_col = df.columns[-1]
df["Category"] = df[dest_col].map(get_route_category)
df_with_lookup = df.merge(lookup_df, on="Category", how="left")
df_with_lookup.head(5)

,FLIGHT_DATE,FLIGHT_TIME,TIME_OF_DAY,AIRLINE_CD,Flight_No,Departure,ARRIVAL_STATION_CD,ARRIVAL_COUNTRY,ARRIVAL_REGION,HAUL,...,FIRST_CLASS_SEATS,BUSINESS_CLASS_SEATS,ECONOMY_SEATS,TIER1_ELIGIBLE_PAX,TIER2_ELIGIBLE_PAX,TIER3_ELIGIBLE_PAX,Category,Tier 1%,Tier 2%,Tier 3%
0,2025-09-02,14:19:00,Afternoon,BA,BA5211,LHR,LAX,USA,North America,LONG,...,8,49,178,0,10,38,Short-haul,5,20,60
1,2025-06-10,06:42:00,Morning,BA,BA7282,LHR,LAX,USA,North America,LONG,...,8,49,178,0,7,28,Short-haul,5,20,60
2,2025-10-27,15:33:00,Afternoon,BA,BA1896,LHR,FRA,Germany,Europe,SHORT,...,0,17,163,0,11,40,Short-haul,5,20,60
3,2025-06-15,18:29:00,Evening,BA,BA5497,LHR,IST,Turkey,Europe,SHORT,...,0,8,172,0,16,54,Short-haul,5,20,60
4,2025-08-25,20:35:00,Evening,BA,BA1493,LHR,FRA,Germany,Europe,SHORT,...,0,13,167,0,6,27,Short-haul,5,20,60


In [44]:
# Build simplified output table: Flight No, Departure, Destination, Category, Tier 1%, Tier 2%, Tier 3%
flight_col = next((c for c in df_with_lookup.columns if "flight" in str(c).lower() or c == "Flight_No"), df_with_lookup.columns[0])
dep_col = next((c for c in df_with_lookup.columns if "depart" in str(c).lower() or c == "Departure"), df_with_lookup.columns[1] if len(df_with_lookup.columns) > 1 else None)

out_cols = [flight_col, dep_col, dest_col, "Category", "Tier 1%", "Tier 2%", "Tier 3%"]
out_cols = [c for c in out_cols if c in df_with_lookup.columns]
result = df_with_lookup[out_cols].copy()
result = result.rename(columns={flight_col: "Flight No.", dep_col: "Departure", dest_col: "Destination"})

# Representative sample: up to 30 flights (e.g. first 30, or use time filter if you have datetime)
SAMPLE_SIZE = 30
sample = result.head(SAMPLE_SIZE)
print("Sample of flights with category and lounge eligibility (%):")
sample

Sample of flights with category and lounge eligibility (%):


,Flight No.,Departure,Destination,Category,Tier 1%,Tier 2%,Tier 3%
0,2025-09-02,LHR,Afternoon,Short-haul,5,20,60
1,2025-06-10,LHR,Morning,Short-haul,5,20,60
2,2025-10-27,LHR,Afternoon,Short-haul,5,20,60
3,2025-06-15,LHR,Evening,Short-haul,5,20,60
4,2025-08-25,LHR,Evening,Short-haul,5,20,60
5,2025-07-12,LHR,Evening,Short-haul,5,20,60
6,2025-09-16,LHR,Evening,Short-haul,5,20,60
7,2025-04-24,LHR,Afternoon,Short-haul,5,20,60
8,2025-05-17,LHR,Lunchtime,Short-haul,5,20,60
9,2025-05-01,LHR,Morning,Short-haul,5,20,60


In [45]:
# Optional: format Departure as time (if it's stored as decimal fraction of day, e.g. 0.291667 = 07:00)
def format_departure(val):
    if pd.isna(val):
        return val
    try:
        v = float(val)
        if 0 <= v < 1:  # fraction of day
            h = int(v * 24)
            m = int((v * 24 - h) * 60)
            return f"{h:02d}:{m:02d}"
    except (TypeError, ValueError):
        pass
    return val

sample_display = sample.copy()
sample_display["Departure"] = sample_display["Departure"].map(format_departure)
sample_display["Tier 1%"] = sample_display["Tier 1%"].astype(int).astype(str) + "%"
sample_display["Tier 2%"] = sample_display["Tier 2%"].astype(int).astype(str) + "%"
sample_display["Tier 3%"] = sample_display["Tier 3%"].astype(int).astype(str) + "%"
sample_display

,Flight No.,Departure,Destination,Category,Tier 1%,Tier 2%,Tier 3%
0,2025-09-02,LHR,Afternoon,Short-haul,5%,20%,60%
1,2025-06-10,LHR,Morning,Short-haul,5%,20%,60%
2,2025-10-27,LHR,Afternoon,Short-haul,5%,20%,60%
3,2025-06-15,LHR,Evening,Short-haul,5%,20%,60%
4,2025-08-25,LHR,Evening,Short-haul,5%,20%,60%
5,2025-07-12,LHR,Evening,Short-haul,5%,20%,60%
6,2025-09-16,LHR,Evening,Short-haul,5%,20%,60%
7,2025-04-24,LHR,Afternoon,Short-haul,5%,20%,60%
8,2025-05-17,LHR,Lunchtime,Short-haul,5%,20%,60%
9,2025-05-01,LHR,Morning,Short-haul,5%,20%,60%


## 4. Reusable lookup table and next steps

The **lookup table** below is the main deliverable: apply it to any future schedule by assigning each flight to a category (e.g. using destination or route type), then use the Tier % to estimate lounge-eligible passengers when you have load factors or passenger counts.

**Note:** Tier 1 (Concorde Room) is treated as hypothetical for Terminal 3 planning—there is currently no Concorde Room at T3; the estimate supports future capacity decisions.

In [46]:
# Final lookup table (reusable by category)
lookup_df_display = lookup_df.copy()
lookup_df_display["Tier 1%"] = lookup_df_display["Tier 1%"].astype(int).astype(str) + "%"
lookup_df_display["Tier 2%"] = lookup_df_display["Tier 2%"].astype(int).astype(str) + "%"
lookup_df_display["Tier 3%"] = lookup_df_display["Tier 3%"].astype(int).astype(str) + "%"
lookup_df_display

,Category,Tier 1%,Tier 2%,Tier 3%
0,Short-haul,5%,20%,60%
1,Mid-haul A,7%,25%,65%
2,Mid-haul B,8%,28%,68%
3,Long-haul,10%,32%,72%
4,Unknown,5%,20%,58%


In [47]:
# Category distribution in the full schedule (to validate groupings)
print("Flights per category:")
df_with_lookup["Category"].value_counts().sort_index()

Flights per category:


Short-haul    10000
Name: Category, dtype: int64

In [48]:
# Standardise column names for use in the rest of the notebook
# Adjust these if your Excel uses different headers (run the previous cell to see actual names)
col_map = {}
for c in df.columns:
    c_lower = str(c).strip().lower()
    if "flight" in c_lower or "number" in c_lower:
        col_map[c] = "Flight_No"
    elif "depart" in c_lower or "dep" in c_lower:
        col_map[c] = "Departure"
    elif "dest" in c_lower or "destination" in c_lower or "to" in c_lower:
        col_map[c] = "Destination"
    elif "origin" in c_lower or "from" in c_lower or "departure" in c_lower and c != "Departure":
        col_map[c] = "Origin"
df = df.rename(columns=col_map)
# Keep only columns we need; add others if present
keep = [c for c in ["Flight_No", "Departure", "Destination", "Origin"] if c in df.columns]
if not keep:
    keep = list(df.columns[: min(5, len(df.columns))])  # fallback: first 5 columns
df = df[[c for c in keep if c in df.columns] or list(df.columns)].copy()
df.head(10)

,Flight_No,Flight_No,Flight_No,Departure
0,2025-09-02,14:19:00,BA5211,LHR
1,2025-06-10,06:42:00,BA7282,LHR
2,2025-10-27,15:33:00,BA1896,LHR
3,2025-06-15,18:29:00,BA5497,LHR
4,2025-08-25,20:35:00,BA1493,LHR
5,2025-07-12,19:08:00,BA4954,LHR
6,2025-09-16,21:44:00,BA9840,LHR
7,2025-04-24,14:50:00,BA7116,LHR
8,2025-05-17,12:13:00,BA4424,LHR
9,2025-05-01,09:55:00,BA4878,LHR
